In [ ]:
def get_band_ds(n1,n2):
    dsh = xr.open_mfdataset(["/data/GPM_HIM8/HIM8_201811.nc","/data/GPM_HIM8/HIM8_201901.nc","/data/GPM_HIM8/HIM8_201902.nc"], combine='by_coords')
    dsg = xr.open_mfdataset(["/data/GPM_HIM8/GPM_201811.nc","/data/GPM_HIM8/GPM_201901.nc","/data/GPM_HIM8/GPM_201902.nc"], combine='by_coords')
    b1 = dsh[f'B{n1}'].values
    b2 = dsh[f'B{n2}'].values
    y_train = dsg['PrecCal'].sel(time=dsh.time).values
    dsg.close()
    dsh.close()
    
    b1_mean = b1.mean()
    b1_std = b1.std()
    b2_mean = b2.mean()
    b2_std = b2.std()
    
    b1 = (b1-b1_mean)/b1_std
    b2 = (b2-b2_mean)/b2_std
    
    x_train = np.stack((b1,b2), axis=-1)
    b1 = None
    b2 = None

    not_nan = ~np.isnan(x_train).any(axis=(1,2,3))
    x_train = x_train[not_nan,:]
    y_train = y_train[not_nan,:,:,None]
    
    dsh = xr.open_dataset("/data/GPM_HIM8/HIM8_201812.nc")
    dsg = xr.open_dataset("/data/GPM_HIM8/GPM_201812.nc")
    b1 = dsh[f'B{n1}'].values
    b2 = dsh[f'B{n2}'].values
    y_test = dsg['PrecCal'].sel(time=dsh.time).values
    dsg.close()
    dsh.close()
    
    b1 = (b1-b1_mean)/b1_std
    b2 = (b2-b2_mean)/b2_std
    
    x_test = np.stack((b1,b2), axis=-1)
    b1 = None
    b2 = None

    not_nan = ~np.isnan(x_test).any(axis=(1,2,3))
    x_test = x_test[not_nan,:]
    y_test = y_test[not_nan,:,:,None]
    
    return x_train, np.clip(y_train,0,30), x_test, np.clip(y_test,0,30)
 

In [4]:
import xarray as xr

def get_band_ds(n):
    dsh = xr.open_mfdataset(["/data/GPM_HIM8/HIM8_201811.nc","/data/GPM_HIM8/HIM8_201901.nc","/data/GPM_HIM8/HIM8_201902.nc"], combine='by_coords')
    dsg = xr.open_mfdataset(["/data/GPM_HIM8/GPM_201811.nc","/data/GPM_HIM8/GPM_201901.nc","/data/GPM_HIM8/GPM_201902.nc"], combine='by_coords')
    
    x_train = dsh[f'B{n}'].values
    y_train = dsg['PrecCal'].sel(time=dsh.time).values
    dsg.close()
    dsh.close()
    
    not_nan = ~np.isnan(x_train).any(axis=(1,2))
    x_train = x_train[not_nan,:,:,None]
    y_train = y_train[not_nan,:,:,None]
    
    dsh = xr.open_dataset("/data/GPM_HIM8/HIM8_201812.nc")
    dsg = xr.open_dataset("/data/GPM_HIM8/GPM_201812.nc")
    
    x_test = dsh[f'B{n}'].values
    y_test = dsg['PrecCal'].sel(time=dsh.time).values
    dsg.close()
    dsh.close()
    
    not_nan = ~np.isnan(x_test).any(axis=(1,2))
    x_test = x_test[not_nan,:,:,None]
    y_test = y_test[not_nan,:,:,None]
    
    return (x_train-x_train.mean())/x_train.std(), np.clip(y_train,0,30), (x_test-x_train.mean())/x_train.std(), np.clip(y_test,0,30)
 

In [5]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.layers import BatchNormalization, Conv2D, UpSampling2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import SGD, Adam, Adagrad
import numpy as np
from tensorflow.keras import backend as K

def get_unet():
    concat_axis = 3
    inputs = layers.Input(shape = (512, 512, 1))

    feats = 4#16
    bn0 = BatchNormalization(axis=3)(inputs)
    conv1 = layers.Conv2D(feats, (3, 3), activation='relu', padding='same', name='conv1_1')(bn0)
    bn2 = BatchNormalization(axis=3)(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(bn2) #256

    conv2 = layers.Conv2D(2*feats, (3, 3), activation='relu', padding='same')(pool1)
    bn4 = BatchNormalization(axis=3)(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(bn4) #128

    conv3 = layers.Conv2D(4*feats, (3, 3), activation='relu', padding='same')(pool2)
    bn6 = BatchNormalization(axis=3)(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(bn6) #64

    conv4 = layers.Conv2D(8*feats, (3, 3), activation='relu', padding='same')(pool3)
    bn8 = BatchNormalization(axis=3)(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(bn8) #32

    conv5 = layers.Conv2D(16*feats, (3, 3), activation='relu', padding='same')(pool4)
    bn10 = BatchNormalization(axis=3)(conv5)
    pool5 = layers.MaxPooling2D(pool_size=(2, 2))(bn10) #16

    conv6 = layers.Conv2D(32*feats, (3, 3), activation='relu', padding='same')(pool5)
    bn11 = BatchNormalization(axis=3)(conv6)

    up_conv6 = layers.UpSampling2D(size=(2, 2))(bn11) #32
    up7 = layers.concatenate([up_conv6, conv5], axis=concat_axis)

    conv7 = layers.Conv2D(16*feats, (3, 3), activation='relu', padding='same')(up7)
    bn13 = BatchNormalization(axis=3)(conv7)
    
    up_conv5 = layers.UpSampling2D(size=(2, 2))(bn13) #64
    up6 = layers.concatenate([up_conv5, conv4], axis=concat_axis)

    conv6 = layers.Conv2D(8*feats, (3, 3), activation='relu', padding='same')(up6)
    bn15 = BatchNormalization(axis=3)(conv6)

    up_conv6 = layers.UpSampling2D(size=(2, 2))(bn15) #128
    up7 = layers.concatenate([up_conv6, conv3], axis=concat_axis)
    
    conv7 = layers.Conv2D(4*feats, (3, 3), activation='relu', padding='same')(up7)
    bn13 = BatchNormalization(axis=3)(conv7)

    # Rectify last convolution layer to constraint output to positive precipitation values.
    conv8 = layers.Conv2D(1, (1, 1), activation='relu')(bn13)

    model = models.Model(inputs=inputs, outputs=conv8)


    return model

model = get_unet()
print(model.summary())
opt = Adagrad(lr=0.0001)
model.compile(loss='mse', optimizer=opt)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
batch_normalization_20 (BatchNo (None, 512, 512, 1)  4           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, 512, 512, 4)  40          batch_normalization_20[0][0]     
__________________________________________________________________________________________________
batch_normalization_21 (BatchNo (None, 512, 512, 4)  16          conv1_1[0][0]                    
____________________________________________________________________________________________

In [ ]:
import pickle
from numpy.random import seed

for b in range(8,17):
    seed(1)
    tf.random.set_seed(2)
    print(f"-----------{b}-----------")
    x_train,y_train,x_test,y_test = get_band_ds(b)
    print(x_train.shape, y_train.shape)

    print("MSE train", np.mean(np.square(y_train)))
    print("MSE test", np.mean(np.square(y_test)))
    
    model = get_unet()
    print(model.summary())
    model.compile(loss='mse', optimizer=Adagrad(lr=0.0001))
    
    history = model.fit(x_train, y_train, validation_data=(x_test, y_test), shuffle=True, epochs=200)
    
    with open(f'history_3months_100epochs_4chan_b{b}.pkl', 'wb') as f:
        pickle.dump(history.history, f)

    model.save(f'model_3months_100epochs_4chan_b{b}.h5')

-----------8-----------
(4229, 512, 512, 1) (4229, 128, 128, 1)
MSE train 0.7034511
MSE test 1.3877556
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
batch_normalization_30 (BatchNo (None, 512, 512, 1)  4           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, 512, 512, 4)  40          batch_normalization_30[0][0]     
__________________________________________________________________________________________________
batch_normalization_31 (BatchNo (None, 512, 512, 4)  16          conv1_1[0][0]          

4229/4229 [==============================] - 13s 3ms/sample - loss: 0.6489 - val_loss: 1.1858
Epoch 10/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.6438 - val_loss: 1.1755
Epoch 11/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.6392 - val_loss: 1.1689
Epoch 12/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.6342 - val_loss: 1.1638
Epoch 13/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.6284 - val_loss: 1.1575
Epoch 14/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.6269 - val_loss: 1.1508
Epoch 15/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.6246 - val_loss: 1.1464
Epoch 16/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.6208 - val_loss: 1.1444
Epoch 17/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.6153 - val_loss: 1.1397
Epoch 18/200
4229/4229 [==========================

4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5566 - val_loss: 1.0566
Epoch 86/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5577 - val_loss: 1.0572
Epoch 87/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5559 - val_loss: 1.0568
Epoch 88/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5577 - val_loss: 1.0573
Epoch 89/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5570 - val_loss: 1.0554
Epoch 90/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5551 - val_loss: 1.0567
Epoch 91/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5589 - val_loss: 1.0565
Epoch 92/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5530 - val_loss: 1.0570
Epoch 93/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5527 - val_loss: 1.0553
Epoch 94/200
4229/4229 [==========================

4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5303 - val_loss: 1.0491
Epoch 162/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5319 - val_loss: 1.0487
Epoch 163/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5301 - val_loss: 1.0488
Epoch 164/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5306 - val_loss: 1.0492
Epoch 165/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5321 - val_loss: 1.0495
Epoch 166/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5319 - val_loss: 1.0493
Epoch 167/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5308 - val_loss: 1.0487
Epoch 168/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5297 - val_loss: 1.0492
Epoch 169/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5319 - val_loss: 1.0499
Epoch 170/200
4229/4229 [=================

Train on 4229 samples, validate on 1483 samples
Epoch 1/200
4229/4229 [==============================] - 15s 3ms/sample - loss: 0.9264 - val_loss: 1.3575
Epoch 2/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.7843 - val_loss: 1.2527
Epoch 3/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.7348 - val_loss: 1.2117
Epoch 4/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.7040 - val_loss: 1.2062
Epoch 5/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.6843 - val_loss: 1.2065
Epoch 6/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.6687 - val_loss: 1.2014
Epoch 7/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.6571 - val_loss: 1.1894
Epoch 8/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.6476 - val_loss: 1.1798
Epoch 9/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.6388 - val_loss: 1.170

4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5480 - val_loss: 1.0462
Epoch 78/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5489 - val_loss: 1.0444
Epoch 79/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5453 - val_loss: 1.0451
Epoch 80/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5435 - val_loss: 1.0444
Epoch 81/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5442 - val_loss: 1.0444
Epoch 82/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5441 - val_loss: 1.0436
Epoch 83/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5420 - val_loss: 1.0447
Epoch 84/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5437 - val_loss: 1.0426
Epoch 85/200
4229/4229 [==============================] - 13s 3ms/sample - loss: 0.5426 - val_loss: 1.0430
Epoch 86/200
4229/4229 [==========================

In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt
#200 1
res = np.array([1.0488,1.0349,1.0106,0.9807,0.9958,0.9834,0.9940,0.9971,1.0128])
#100 1
#res = np.array([1.0371,1.0770,1.0187,0.9976,1.0232,1.0022,1.0058,1.0338,1.0162])

res -= res.min()
res /= res.max()

#plt.figure(figsize=(20,10))
plt.bar(list(range(8,17)),res,tick_label=["B8","B9","B10","B11","B12","B13","B14","B15","B16"])
plt.title("Relative error: Himawari8 Band -> GPM")